In [2]:
import os
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
# from torchviz import make_dot
from sklearn.preprocessing import StandardScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class PHQDataset(Dataset):
    def __init__(self, csv_file, data_folder, max_seq_length=None, feature_type=None):
        self.data_folder = data_folder
        self.data_info = pd.read_csv(csv_file)
        self.max_seq_length = max_seq_length
        self.feature_type = feature_type
        self.scaler = StandardScaler()

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        participant_id = self.data_info.iloc[idx]['Participant_ID']
        phq_score = self.data_info.iloc[idx]['PHQ_Score']
        filepath = os.path.join(self.data_folder, f"{participant_id}_OpenFace2.1.0_Pose_gaze_AUs.csv")
        features_df = pd.read_csv(filepath)
        features_df = features_df.iloc[:, 2:]  # Remove the first two columns (frame and timestamp)
        # features = pd.read_csv(filepath).to_numpy()
        # features = features[:, 2:] # Remove the first two features (frame and timestamp)

        # Define column sets
        pose_columns = [col for col in features_df.columns if col.startswith('pose_')]
        gaze_columns = [col for col in features_df.columns if col.startswith('gaze_')]
        au_r_columns = [col for col in features_df.columns if col.endswith('_r')]
        au_c_columns = [col for col in features_df.columns if col.endswith('_c')]

        # Select columns based on feature type
        if self.feature_type == 'pose':
            selected_columns = pose_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'gaze':
            selected_columns = gaze_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'au_r':
            selected_columns = au_r_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'au_c':
            selected_columns = au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'confidence_success_pose':
            selected_columns = pose_columns + ['confidence', 'success']
        elif self.feature_type == 'confidence_success_gaze':
            selected_columns = gaze_columns + ['confidence', 'success']
        elif self.feature_type == 'confidence_success_AUintens':
            selected_columns = au_r_columns + ['confidence', 'success']
        elif self.feature_type == 'confidence_success_AUoccurr':
            selected_columns = au_c_columns + ['confidence', 'success']
        elif self.feature_type == 'confidence_success_AUoccurr_pose_gaze':
            selected_columns = au_c_columns + gaze_columns + pose_columns + ['confidence', 'success']
        elif self.feature_type == 'confidence_success_AUintens_pose_gaze':
            selected_columns = au_r_columns + gaze_columns + pose_columns + ['confidence', 'success']
        elif self.feature_type == 'pose_gaze':
            selected_columns = gaze_columns + pose_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'pose_au_r':
            selected_columns = pose_columns + au_r_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'pose_au_c':
            selected_columns = pose_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'gaze_au_r':
            selected_columns = gaze_columns + au_r_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'gaze_au_c':
            selected_columns = gaze_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'au_r_au_c':
            selected_columns = au_r_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'pose_gaze_au_r':
            selected_columns = pose_columns + gaze_columns + au_r_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'pose_gaze_au_c':
            selected_columns = pose_columns + gaze_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'pose_au_r_au_c':
            selected_columns = pose_columns + au_r_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'gaze_au_r_au_c':
            selected_columns = gaze_columns + au_r_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'all':
            selected_columns = pose_columns + gaze_columns + au_r_columns + au_c_columns
            # Filter by confidence
            indices_to_remove = features_df[features_df['confidence'] < 0.9].index
            features_df = features_df.drop(index=indices_to_remove)
        elif self.feature_type == 'confidence_success_all':
            selected_columns = None
        else:
            selected_columns = None  # Use all columns if feature_type is not recognized
        
        if selected_columns is not None:
            features_df = features_df[selected_columns]
            # Remove rows with NaN values
            features_df = features_df.dropna()
            features = features_df.values
        else:
            # Remove rows with NaN values
            features_df = features_df.dropna()
            features = features_df.values
        
        # Apply feature normalization
        # features_normalized = self.scaler.fit_transform(features)
        # print("features_normalized shape: ", features_normalized.shape)

        # if self.max_seq_length is not None:
        #     padded_features = np.zeros((self.max_seq_length, features_normalized.shape[1]))
        #     padded_features[:features_normalized.shape[0], :features_normalized.shape[1]] = features_normalized
        #     features_normalized = padded_features

        # print('features.shape at the end: ', features.shape)
        return torch.tensor(features, dtype=torch.float32), torch.tensor(phq_score, dtype=torch.float32)
        # return torch.tensor(features_normalized, dtype=torch.float32), torch.tensor(phq_score, dtype=torch.float32)

class Attention(nn.Module):
    def __init__(self, feature_dim, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.proj = nn.Linear(feature_dim, 64)
        self.context_vector = nn.Linear(64, 1, bias=False)

    def forward(self, x):
        x_proj = torch.tanh(self.proj(x))
        context_vector = self.context_vector(x_proj).squeeze(2)
        attention_weights = torch.softmax(context_vector, dim=1)
        weighted = torch.mul(x, attention_weights.unsqueeze(-1).expand_as(x))
        return torch.sum(weighted, dim=1)
    
# class EnhancedPHQLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers=2, dropout_rate=0.5):
#         super(EnhancedPHQLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
#                             batch_first=True, bidirectional=True, dropout=dropout_rate)
#         self.attention = Attention(hidden_size * 2)
#         self.fc = nn.Linear(hidden_size * 2, 1)

#     def forward(self, x):
#         lstm_out, _ = self.lstm(x)
#         attn_out = self.attention(lstm_out)
#         # final_output = self.fc(lstm_out[:, -1, :])  # Taking the last output of the sequence to check the performance without attention layer
#         final_output = self.fc(attn_out)
#         return final_output

class EnhancedPHQLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2, dropout_rate=0.5):
        super(EnhancedPHQLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.attention = Attention(hidden_size * 2)
        self.fc = nn.Linear(hidden_size * 2, 128)  # Adjust the output size of the fully connected layer

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_out = self.attention(lstm_out)
        return attn_out  # Return the output of the attention layer

    def get_last_fc_output(self, x):
        lstm_out, _ = self.lstm(x)
        # print("Shape of lstm_out:", lstm_out.shape)
        attn_out = self.attention(lstm_out)
        print("Shape of attn_out:", attn_out.shape)
        fc_out = self.fc(attn_out)
        # print("Shape of fc_out:", fc_out.shape)
        
        # final_output_a = self.fc(lstm_out[:, -1, :]) 
        # print("Shape of final_output_a:", final_output_a.shape)
        # final_output_b = self.fc(attn_out)
        # print("Shape of final_output_b:", final_output_b.shape)
        # return fc_out
        return attn_out


criterion = nn.MSELoss()
mae = nn.L1Loss()

def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_stack = torch.stack(yy, dim=0)
    return xx_pad, yy_stack, x_lens

# Define different combinations of feature types
feature_combinations = ['all', 'pose','gaze','au_r','au_c', \
                        'pose_gaze', 'pose_au_r', 'pose_au_c', \
                        'gaze_au_r', 'gaze_au_c', \
                        'au_r_au_c', \
                        'pose_gaze_au_r', 'pose_gaze_au_c', 'pose_au_r_au_c', 'gaze_au_r_au_c', \
                        'confidence_success_all', \
                        'confidence_success_AUintens', 'confidence_success_AUoccurr', \
                        'confidence_success_pose','confidence_success_gaze', \
                        'confidence_success_AUintens_pose_gaze', \
                        'confidence_success_AUoccurr_pose_gaze']


Using device: cpu


In [3]:
import os
import shutil
import pandas as pd

openface_data_path = "/Users/misha/My_Projects/DAIC_Depression/depression-detection/data/DAIC_openface_features/"
labels_path = "/Users/misha/My_Projects/DAIC_Depression/depression-detection/data/labels/"

# Read labels
train_labels = pd.read_csv(os.path.join(labels_path, 'train_split.csv'))
dev_labels = pd.read_csv(os.path.join(labels_path, 'dev_split.csv'))
test_labels = pd.read_csv(os.path.join(labels_path, 'test_split.csv'))

# Convert labels to NumPy arrays
train_ids = train_labels['Participant_ID'].values
dev_ids = dev_labels['Participant_ID'].values
test_ids = test_labels['Participant_ID'].values


In [4]:
train_data_path = os.path.join(openface_data_path, "train")
dev_data_path = os.path.join(openface_data_path, "dev")
test_data_path = os.path.join(openface_data_path, "test")

train_dataset = PHQDataset(os.path.join(labels_path, 'train_split.csv'), train_data_path, feature_type="all")
dev_dataset = PHQDataset(os.path.join(labels_path, 'dev_split.csv'), dev_data_path, feature_type="all")
test_dataset = PHQDataset(os.path.join(labels_path, 'test_split.csv'), test_data_path, feature_type="all")

test_dataset

In [5]:
import torch

# Load the pretrained model
model_path = "lstm_model_all.pth"  # Update the path if necessary
pretrained_model = torch.load(model_path, map_location=torch.device('cpu'))

print("Model layers:")
for name, layer in pretrained_model.named_children():
    print(name, layer)

Model layers:
lstm LSTM(49, 64, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
attention Attention(
  (proj): Linear(in_features=128, out_features=64, bias=True)
  (context_vector): Linear(in_features=64, out_features=1, bias=False)
)
fc Linear(in_features=128, out_features=1, bias=True)


In [6]:
# Assuming you have already defined your datasets (train_dataset, test_dataset, dev_dataset)

# # Function to extract features from a dataset using the pretrained model
# def extract_features_from_dataset(model, dataset):
#     extracted_features = []
#     model.eval()
#     with torch.no_grad():
#         for features, phq_scores in dataset:
#             features = features.unsqueeze(0).to(device)  # Add batch dimension if needed
#             output = model(features).squeeze().cpu().numpy()  # Forward pass through the model
#             extracted_features.append(output)
#     return extracted_features

def extract_features_from_dataset(model, dataset):
    i = 0
    extracted_features = []
    model.eval()
    with torch.no_grad():
        for features, phq_scores in dataset:
            # print("features: ", features)
            # print("phq_scores: ", phq_scores)
            # print("i: ", i)
            features = features.unsqueeze(0).to(device)  # Add batch dimension if needed
            output = model.get_last_fc_output(features).squeeze().cpu().numpy()  # Get output from last fully connected layer
            # print("output shape: ", output.shape)
            extracted_features.append(output)
            i += 1
    return extracted_features


# Extract features from train, test, and dev sets
train_features = extract_features_from_dataset(pretrained_model, train_dataset)
test_features = extract_features_from_dataset(pretrained_model, test_dataset)
dev_features = extract_features_from_dataset(pretrained_model, dev_dataset)

# Now train_features, test_features, and dev_features contain the extracted features for the respective datasets


Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])
Shape of attn_out: torch.Size([1, 128])


KeyboardInterrupt: 